In [4]:

!pip install -q m2cgen

# Step 2: Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import m2cgen as m2c
import os

# Step 3: Load your datA
df = pd.read_csv('/kaggle/input/senosr-calib/timezones_handled.csv')
# For demo, assuming df is already loaded

# Step 4: Prepare features and target
X = df[['dht11_temp']]   # input: raw DHT11 temperature
Y = df['avg_temp']       # target: corrected average temperature

# Step 5: Split into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Step 6: Create and train Random Forest model
rf = RandomForestRegressor(n_estimators=20, random_state=42)
rf.fit(X_train, Y_train)

# Step 7: Make predictions on test set
y_corrected = rf.predict(X_test)

# Step 8: Evaluate performance
mae = mean_absolute_error(Y_test, y_corrected)
rmse = mean_squared_error(Y_test, y_corrected, squared=False)
r2 = r2_score(Y_test, y_corrected)

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R^2 score: {r2:.4f}")

# Step 9: Export model to C code and save as model.c
output_path = "/kaggle/working/model.c"
with open(output_path, "w") as f:
    f.write(c_code)
print(f"✅ Model saved to {output_path}")


# Step 10: Show the size of the exported C file
size_kb = os.path.getsize("model.c") / 1024
print(f"✅ Model exported as 'model.c' (Size: {size_kb:.2f} KB)")

# Optional: Step 11: Plot predictions vs actuals
plt.figure(figsize=(8,6))
plt.scatter(Y_test, y_corrected, alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--')
plt.xlabel("Actual Average Temperature")
plt.ylabel("Corrected DHT11 Temperature")
plt.title("Predicted vs Actual (20 Trees)")
plt.grid(True)
plt.show()


In [ ]:
#melvin
#export this code to  arduino  ide and upload



#include <stdio.h>
#include <stdlib.h>  // for atof
#include "model.c"   // your exported model

void app_main(void) {
    char input_line[64];
    while (1) {
        printf("Enter raw DHT11 temperature (or 'q' to quit): ");

        // Read a line from UART (serial console)
        if (fgets(input_line, sizeof(input_line), stdin) == NULL) {
            printf("\nError reading input\n");
            continue;
        }

        // Check for quit command
        if (input_line[0] == 'q' || input_line[0] == 'Q') {
            printf("Exiting...\n");
            break;
        }

        // Convert input string to double
        double raw_temp = atof(input_line);

        // Prepare input array for model (1 feature)
        double model_input[1] = { raw_temp };

        // Run the prediction
        double calibrated_temp = score(model_input);

        // Print results
        printf("Raw DHT11 temperature: %.2f\n", raw_temp);
        printf("Calibrated temperature: %.2f\n\n", calibrated_temp);
    }
}

